<a href="https://colab.research.google.com/github/sergak0/AIIJC/blob/main/Lisa/nb_u_re.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Зависимости



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pymorphy2[fast]

In [ ]:
import random
import re
import string
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from typing import List
from sklearn.metrics import classification_report
import pickle
from typing import List
import nltk
import string
import pymorphy2
import codecs
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create our model

In [ ]:
class Preparator():
    def __init__(self,
                 preparator_fold_name: str = '/content/drive/MyDrive/aiijc_sber/'):
        stopwords_path = preparator_fold_name + 'all_stopwords.txt'
        bigram_model_path = preparator_fold_name + 'bigram_model.pkl'
        self.morph = pymorphy2.MorphAnalyzer()
        self.tokenizer = nltk.WordPunctTokenizer()
        self.stopwords = set(line.strip() for line in codecs.open(stopwords_path, "r", "utf_8_sig").readlines())
        self.bigram_mod = gensim.models.Phrases.load(bigram_model_path)

    def prepare_corp(self, news_list: List[str]):
        return [self.newstext2token(news_text) for news_text in news_list]

    def newstext2token(self, news_text: str):
        tokens = self.tokenizer.tokenize(news_text.lower())
        tokens_with_no_punct = [self.morph.parse(w)[0].normal_form for w in tokens if
                                all(c in 'йцукенгшщзхъёфывапролджэячсмитьбю' for c in w)]
        tokens_base_forms = [w for w in tokens_with_no_punct if w not in self.stopwords]
        tokens_last = [w for w in tokens_base_forms if len(w) > 1]
        tokens_bigrammed = self.make_bigrams(tokens_last)
        return ' '.join(tokens_bigrammed)

    def make_bigrams(self, doc):
        return self.bigram_mod[doc]

    def prepare_texts(self, texts):
        return self.prepare_corp(texts)



In [ ]:
class BayesModel():

    def __init__(self,
                 pretrained=True,
                 nb_fold_name: str = '/content/drive/MyDrive/aiijc_sber/nb/'):

        if not pretrained:
            self.vect = CountVectorizer()
            self.tfidf = TfidfTransformer()
            self.clf = MultinomialNB()
        else:
            self.vect = pickle.load(open(nb_fold_name + 'nb_vect.sav', 'rb'))
            self.tfidf = pickle.load(open(nb_fold_name + 'nb_tfidf.sav', 'rb'))
            self.clf = pickle.load(open(nb_fold_name + 'nb_clf.sav', 'rb'))

        self.nb = Pipeline([('vect', self.vect),
                            ('tfidf', self.tfidf),
                            ('clf', self.clf),
                            ])

        self.classes = ['животные', "музыка", "спорт", "литература"]

    def fit(self, X_train, Y_train):
        self.nb.fit(X_train, Y_train)

    def predict(self, X_test: List[str]):
        return self.nb.predict(X_test)

    def predict_proba(self, X_test: List[str]):
        return self.nb.predict_proba(X_test)

    def show_metrics(self, X_test, Y_test):
        y_pred = self.predict(X_test)
        print('accuracy %s' % accuracy_score(y_pred, Y_test))
        print(classification_report(Y_test, y_pred, target_names=self.classes))

    def score(self, X_test, Y_test):
        return self.nb.score(X_test, Y_test)

    def save_models(self,
                    nb_fold_name: str = '/content/drive/MyDrive/aiijc_sber/nb/'):
        pickle.dump(self.vect, open(nb_fold_name + "nb_vect.sav", 'wb'))
        pickle.dump(self.tfidf, open(nb_fold_name + 'nb_tfidf.sav', 'wb'))
        pickle.dump(self.clf, open(nb_fold_name + 'nb_clf.sav', 'wb'))


In [ ]:
nb_model = BayesModel(pretrained=False)

# Concat the data we downloaded and marked

In [ ]:
all_data = pd.read_csv('/content/drive/MyDrive/aiijc_sber/tokens_full_cleaned.csv')
all_data.head()

,Unnamed: 0,texts,y,tokens
0,0,Живо́тные (лат. Animalia) — традиционно (со вр...,0,живо тный лата аристотель выделять организм би...
1,1,Пти́цы (лат. Aves) — группа теплокровных яйцек...,0,пти цы лата группа теплокровный яйцекладущий п...
2,2,Союз охраны птиц России (СОПР) — общероссийска...,0,союз охрана птица россия сопра общероссийский ...
3,3,Крючковидные отростки рёбер — костяные выросты...,0,крючковидный отросток ребро костяной вырост ве...
4,4,Myxozoa (лат.) — класс или подтип паразитическ...,0,лата класс подтип паразитический беспозвоночны...


In [ ]:
dop = pd.read_csv('/content/drive/MyDrive/aiijc_sber/full_marked_dataset.csv')
dop = dop[dop.ans!='неизвестно']
dop.head()

,Unnamed: 0,text,ans
0,0,\t1.1. Летом в спортивный лагерь ходили 50 дет...,спорт
1,1,\t1.2. На концерте в летнем лагере ребята игра...,музыка
3,3,\t1.4. Во время летних соревнований по плавани...,спорт
4,4,\t1.5. В спортивную школу во время летних кани...,спорт
6,6,\t1.7. Самые крупные животные на нашей планете...,животные


In [ ]:
facts2 = pd.read_csv('/content/drive/MyDrive/aiijc_sber/facts2.csv')
mit_dict = {0:"животные",
            1: "музыка",
            2: "спорт",
            3: "литература"}
facts2.labels = [mit_dict[i] for i in facts2.labels]
facts2.head()

,texts,labels
0,1 Хамелеоны могут двигать глазами в разных нап...,животные
1,2 Белка – лучший садовник. Миллионы деревьев в...,животные
2,3 Зуб слона может весить до девяти килограмм!,животные
3,"4 У млекопитающих кровь красная, у насекомых ж...",животные
4,5 В среднем коровы какают 16 раз в день.,животные


In [ ]:
data_texts = all_data.texts.to_list() + dop.text.to_list() + facts2.texts.to_list()
data_ans = [['животные', "музыка", "спорт", "литература"][i] for i in all_data.y] + dop.ans.to_list() + facts2.labels.to_list()
data = pd.DataFrame()
data['text'] = data_texts
data['ans'] = data_ans
data.head()

,text,ans
0,Живо́тные (лат. Animalia) — традиционно (со вр...,животные
1,Пти́цы (лат. Aves) — группа теплокровных яйцек...,животные
2,Союз охраны птиц России (СОПР) — общероссийска...,животные
3,Крючковидные отростки рёбер — костяные выросты...,животные
4,Myxozoa (лат.) — класс или подтип паразитическ...,животные


In [ ]:
data.tail()

,text,ans
3653,Гимн Лиги чемпионов пришел в каждый дом благод...,спорт
3654,"Далеко не каждый знает, что самый популярный в...",спорт
3655,"Когда этот вид спорта только зарождался, то сп...",спорт
3656,"Первым спортсменом, позволившим себе нарушить ...",спорт
3657,Многими любимая стендовая стрельба раньше была...,спорт


# Fut the model

In [ ]:
nb_model = BayesModel(pretrained=False)
nb_model.fit(Preparator().prepare_texts(data.text), data.ans)

In [ ]:
tests = pd.read_csv('/content/drive/MyDrive/aiijc_sber/marked_test.csv')
tests.head()

,Unnamed: 0,task,ans
0,0,Вопрос 1 Денис готовится провести соревнования...,спорт
1,1,Реши задачу.\r\nСколько коробок корма для соба...,животные
2,2,"Если групп элементов несколько, то применяется...",спорт
3,3,Вопрос1 ...,животные
4,4,Для наглядной геометрической иллюстрации объём...,спорт


# Score it

In [ ]:
nb_model.show_metrics(Preparator().prepare_texts(tests.task), tests.ans)

accuracy 0.6517509727626459
              precision    recall  f1-score   support

    животные       0.48      0.75      0.58       121
      музыка       0.54      0.63      0.58       126
       спорт       0.69      0.46      0.55        24
  литература       0.95      0.63      0.76       243

    accuracy                           0.65       514
   macro avg       0.66      0.62      0.62       514
weighted avg       0.73      0.65      0.67       514



# Create submission file

In [ ]:
predictions = nb_model.predict(Preparator().prepare_texts(tests.task))

In [ ]:
tests['category'] = predictions

In [ ]:
def text2vec(tokens: List[str], embeddings: KeyedVectors) -> np.ndarray:
    relevant = 0
    words_vecs = np.zeros((embeddings.vector_size,))
    for word in tokens:
        if word in embeddings:
            words_vecs += embeddings[word]
            relevant += 1

    if relevant:
        words_vecs /= relevant
    return words_vecs
def extract_keywords(tokens: List[str],
                     embeddings: KeyedVectors,
                     top_n: int = 5,
                     diversity: float = 0.5,
                     nr_candidates: int = 20) -> str:
    candidates = []
    doc_embedding = text2vec(tokens, embeddings).reshape(1, -1)
    candidate_embeddings = []
    for candidate in tokens:
        if candidate in embeddings:
            candidates.append(candidate)
            candidate_embeddings.append(embeddings[candidate])

    if len(candidates) == 0:
        return tokens

    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]][::-1]

    return '; '.join(set(keywords))




In [ ]:
tests.head()

,id,task,category
0,0,Вопрос 1 Денис готовится провести соревнования...,спорт
1,1,Реши задачу.\nСколько коробок корма для собак ...,музыка
2,2,"Если групп элементов несколько, то применяется...",спорт
3,3,Вопрос1 ...,животные
4,4,Для наглядной геометрической иллюстрации объём...,спорт


In [ ]:
vect_model = KeyedVectors.load("/content/drive/MyDrive/aiijc_sber/keyword_w2v.kv")

In [ ]:
highlights = [extract_keywords(toks.split(), vect_model, 10) for toks in Preparator().prepare_texts(tests.task)]
tests['keywords'] = highlights

In [ ]:
tests.head()

,id,task,category,keywords
0,0,Вопрос 1 Денис готовится провести соревнования...,спорт,специалист; недоставать; решать; принтер; наст...
1,1,Реши задачу.\nСколько коробок корма для собак ...,музыка,задача; собака; ящик; решить; корм
2,2,"Если групп элементов несколько, то применяется...",спорт,волейбол; сложение; группа; зачёт
3,3,Вопрос1 ...,животные,услуга; покупка; расход; сумма; деньга; потрат...
4,4,Для наглядной геометрической иллюстрации объём...,спорт,пересекаться; круг; увлекаться; наглядный; эйлер


In [ ]:
tests = tests.drop(['task'], axis = 1)
tests.head()

,id,category,keywords
0,0,спорт,специалист; недоставать; решать; принтер; наст...
1,1,музыка,задача; собака; ящик; решить; корм
2,2,спорт,волейбол; сложение; группа; зачёт
3,3,животные,услуга; покупка; расход; сумма; деньга; потрат...
4,4,спорт,пересекаться; круг; увлекаться; наглядный; эйлер


In [ ]:
tests.to_csv('/content/drive/MyDrive/aiijc_sber/test_submission.csv')